In [ ]:
import copy
import random
import prior
import matplotlib.pyplot as plt

from PIL import Image
from ai2thor.controller import Controller

In [ ]:
random.seed(0)

## Acquire pre-constructed dataset
Datasets are located in `~/.prior/`

In [ ]:
dataset = prior.load_dataset("procthor-10k")
dataset

### Load a house

In [ ]:
house = dataset["train"][3]
type(house), house.keys()

In [ ]:
house["doors"]

### Visualize part of the house

In [ ]:
controller = Controller(scene=house, headless=True, start_unity=False, width=500, height=500)

In [ ]:
Image.fromarray(controller.last_event.frame)

In [ ]:
event = controller.step(action="LookUp")
Image.fromarray(event.frame)

In [ ]:
event = controller.step(action="RotateRight")
Image.fromarray(event.frame)

In [ ]:
event = controller.step(action="RotateRight")
Image.fromarray(event.frame)

In [ ]:
event = controller.step(action="RotateRight")
Image.fromarray(event.frame)

In [ ]:
event = controller.step(action="RotateRight")
Image.fromarray(event.frame)

### Visualize top-down map

In [ ]:
def get_top_down_frame():
    # Setup the top-down camera
    event = controller.step(action="GetMapViewCameraProperties", raise_for_failure=True)
    pose = copy.deepcopy(event.metadata["actionReturn"])

    bounds = event.metadata["sceneBounds"]["size"]
    max_bound = max(bounds["x"], bounds["z"])

    pose["fieldOfView"] = 50
    pose["position"]["y"] += 1.1 * max_bound
    pose["orthographic"] = False
    pose["farClippingPlane"] = 50
    del pose["orthographicSize"]

    # add the camera to the scene
    event = controller.step(
        action="AddThirdPartyCamera",
        **pose,
        skyboxColor="white",
        raise_for_failure=True,
    )
    top_down_frame = event.third_party_camera_frames[-1]
    return Image.fromarray(top_down_frame)

In [ ]:
get_top_down_frame()

### Get all reachable positions

In [ ]:
event = controller.step(action="GetReachablePositions")
event.metadata["actionReturn"]

In [ ]:
reachable_positions = event.metadata["actionReturn"]
xs = [rp["x"] for rp in reachable_positions]
zs = [rp["z"] for rp in reachable_positions]

fig, ax = plt.subplots(1, 1)
ax.scatter(xs, zs)
ax.set_xlabel("$x$")
ax.set_ylabel("$z$")
ax.set_title("Reachable Positions in the Scene")
ax.set_aspect("equal")

### Teleport agent to different positions in the environment

In [ ]:
position = random.choice(reachable_positions)
rotation = random.choice(range(360))
print("Teleporting the agent to", position, " with rotation", rotation)

event = controller.step(action="Teleport", position=position, rotation=rotation)
Image.fromarray(event.frame)

In [ ]:
event = controller.step(action="Teleport", position=position, rotation=rotation + 30)
Image.fromarray(event.frame)

In [ ]:
event = controller.step(action="Teleport", position=position, rotation=rotation + 60)
Image.fromarray(event.frame)

In [ ]:
event = controller.step(action="Teleport", position=position, rotation=rotation + 90)
Image.fromarray(event.frame)

### Switch house

In [ ]:
new_house = dataset["train"][1]
controller.reset(scene=new_house)

get_top_down_frame()